In [1]:
import numpy as np 
import re 
import os 
import pandas as pd
from nltk.tokenize import RegexpTokenizer , sent_tokenize
from urllib.request import urlopen
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests
import urllib.request,sys,time ,requests

In [19]:
stopWordsFile = './StopWords/StopWords_Generic.txt'
positiveFile = './MasterDictionary/positive-words.txt'
negativeFile = './MasterDictionary/negative-words.txt'

In [20]:
input = pd.read_excel("Input.xlsx")
input

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...
...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...
111,51844.6,https://insights.blackcoffer.com/what-are-the-...
112,52306.4,https://insights.blackcoffer.com/marketing-dri...


In [21]:
def get_article_names(urls):
  titles = []
  for i in range (len(urls)):
    title = urls[i]
    title_clean = title[title.index( "m/" ) + 2 :-1]. replace('-' , ' ')
    titles.append(title_clean)
  return titles

urls =input["URL"]
urlsTitleDF = get_article_names(urls)
urlsTitleDF


['rise of telemedicine and its impact on livelihood by 2040 3 2',
 'rise of e health and its impact on humans by the year 2030',
 'rise of e health and its imapct on humans by the year 2030 2',
 'rise of telemedicine and its impact on livelihood by 2040 2',
 'rise of telemedicine and its impact on livelihood by 2040 2 2',
 'rise of chatbots and its impact on customer support by the year 2040',
 'rise of e health and its imapct on humans by the year 2030',
 'how does marketing influence businesses and consumers',
 'how advertisement increase your market value',
 'negative effects of marketing on society',
 'how advertisement marketing affects business',
 'rising it cities will impact the economy environment infrastructure and city life by the year 2035',
 'rise of ott platform and its impact on entertainment industry by the year 2030',
 'rise of electric vehicles and its impact on livelihood by 2040',
 'rise of electric vehicle and its impact on livelihood by the year 2040',
 'oil price

In [22]:
url = "https://insights.blackcoffer.com/difference-between-artificial-intelligence-machine-learning-statistics-and-data-mining/"

page=requests.get(url , headers={"User-Agent": "XY"})  
soup = BeautifulSoup(page.text , 'html.parser')
#get title
title = soup . find("h1",attrs = { 'class' : 'entry-title'}).get_text()

#get article text
text = soup . find(attrs = { 'class' : 'td-post-content'}).get_text()
# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)
text


'“Data is the new oil” has become the most important trendline of the 21st century. The reason for this is the advancements in the fields related to data analysis. The field of AI Machine Learning Statistics and Data Mining all deal with data and are developing at such a staggering pace, that these fields have become the most popular buzzwords these days.\nBuzzwords are originated through technical terms but often the underlying essence is ignored through fashionable use and mainly used to impress. This is the main reason for the misconception amongst people. AI, ML, Stats, Data Mining, and many other fields related to the analysis of data are most often mistook for one and the same thing, thus all these words are often used interchangeably to convey one and the same thing. But this is not true at all!\nThe only similarity between these disciplines is that all of these disciplines are related to the analysis of the data and converting this data into “information”.\nIn academics, while 

In [24]:
# Loading positive words
with open(positiveFile,'r') as posfile:
    positivewords=posfile.read().lower()
positiveWordList=positivewords.split('\n')


# Loading negative words
with open(negativeFile ,'r' ,  encoding="ISO-8859-1") as negfile:
    negativeword=negfile.read().lower()
negativeWordList=negativeword.split('\n')

#Loading stop words dictionary for removing stop words
with open(stopWordsFile ,'r') as stop_words:
    stopWords = stop_words.read().lower()
stopWordList = stopWords.split('\n')
stopWordList[-1:] = []



display( positiveWordList[:6]  , negativeWordList[:6] , stopWordList[:6])

['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable']

['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably']

['about', 'above', 'after', 'again', 'all', 'am']

In [43]:

#tokenizeing module and filtering tokens using stop words list, removing punctuations
def tokenizer(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = list(filter(lambda token: token not in stopWordList, tokens))
    return filtered_words

def positive_score (text):
  posword=0
  tokenphrase = tokenizer(text)
  for word in tokenphrase :
    if word in positiveWordList:
       posword+=1
      
    retpos = posword
    return retpos 

def negative_score (text):
  negword=0
  tokenphrase = tokenizer(text)
  for word in tokenphrase :
    if word in negativeWordList : negword +=1

    retneg = negword 
    return retneg

def polarity_score (positive_score , negative_score) :
  return (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
#################################################
def total_word_count(text):
    tokens = tokenizer(text)
    return len(tokens)
#############################################
def AverageSentenceLenght (text):
  Wordcount = len(tokenizer (text))
  SentenceCount = len (sent_tokenize(text))
  if SentenceCount > 0 : Average_Sentence_Lenght = Wordcount / SentenceCount

  avg = Average_Sentence_Lenght

  return round(avg)


# Counting complex words
def complex_word_count(text):
    tokens = tokenizer(text)
    complexWord = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    return complexWord

def percentage_complex_word(text):
    tokens = tokenizer(text)
    complexWord = 0
    complex_word_percentage = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    if len(tokens) != 0:
        complex_word_percentage = complexWord/len(tokens)
    
    return complex_word_percentage

def fog_index(averageSentenceLength, percentageComplexWord):
    fogIndex = 0.4 * (averageSentenceLength + percentageComplexWord)
    return fogIndex






In [26]:
URLS = input ["URL"]
URLS

0      https://insights.blackcoffer.com/is-telehealth...
1      https://insights.blackcoffer.com/how-telehealt...
2      https://insights.blackcoffer.com/is-telemedici...
3      https://insights.blackcoffer.com/is-telehealth...
4      https://insights.blackcoffer.com/how-people-di...
                             ...                        
145    https://insights.blackcoffer.com/blockchain-fo...
146    https://insights.blackcoffer.com/the-future-of...
147    https://insights.blackcoffer.com/big-data-anal...
148    https://insights.blackcoffer.com/business-anal...
149    https://insights.blackcoffer.com/challenges-an...
Name: URL, Length: 150, dtype: object

In [27]:
corps =[]
for url in URLS:

  page=requests.get(url , headers={"User-Agent": "XY"})  
  soup = BeautifulSoup(page.text , 'html.parser')
  #get title
  title = soup . find("h1",attrs = { 'class' : 'entry-title'}).get_text()

  #get article text
  text = soup . find(attrs = { 'class' : 'td-post-content'}).get_text()
  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in text.splitlines())
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  # drop blank lines
  text = '\n'.join(chunk for chunk in chunks if chunk)
  corps.append(text)

  


In [48]:
#df = pd.DataFrame(corps,urlsTitleDF)
df = pd.DataFrame({'title':urlsTitleDF,'corps': corps})
df["total word count"] = df["corps"] . apply (total_word_count)
df["percentage_complex_word"] = df["corps"] . apply (percentage_complex_word)
df["complex_word_count"] = df["corps"] . apply (complex_word_count)
df["AverageSentenceLenght"] = df["corps"] . apply (AverageSentenceLenght)
df["positive_score"] = df["corps"] . apply (positive_score)
df["negative_score"] = df["corps"] . apply (negative_score)
df["polarity_score"] = np.vectorize(polarity_score)(df['positive_score'],df['negative_score'])

df

,title,corps,total word count,percentage_complex_word,complex_word_count,AverageSentenceLenght,positive_score,negative_score,polarity_score
0,is telehealth the future of healthcare 3,Covid-19 has paved the way for advancements in...,437,0.459954,201,17,0,0,0.0
1,how telehealth and telemedicine helping people...,The last 2 years are the most critical time in...,521,0.454894,237,10,0,0,0.0
2,is telemedicine effective in treating patients,"Telemedicine, which allows patients and doctor...",1126,0.475133,535,12,0,0,0.0
3,is telehealth the future of healthcare 2,Telemedicine refers to a specific set of clini...,1210,0.456198,552,14,0,0,0.0
4,how people diverted to telehealth services and...,The future of telehealth stays positive\nTo as...,1141,0.449606,513,14,0,0,0.0
...,...,...,...,...,...,...,...,...,...
145,blockchain for payments,Reconciling with the financial realities of an...,614,0.306189,188,13,0,0,0.0
146,the future of investing,What Is an Investment?\nAn investment is a res...,1060,0.346226,367,17,0,0,0.0
147,big data analytics in healthcare,Quality and affordable healthcare is a vision ...,745,0.417450,311,11,0,0,0.0
148,business analytics in the healthcare industry,Analytics is a statistical scientific process ...,443,0.492099,218,15,0,0,0.0


In [50]:
final = df.drop("corps" , 1)
final 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,title,total word count,percentage_complex_word,complex_word_count,AverageSentenceLenght,positive_score,negative_score,polarity_score
0,is telehealth the future of healthcare 3,437,0.459954,201,17,0,0,0.0
1,how telehealth and telemedicine helping people...,521,0.454894,237,10,0,0,0.0
2,is telemedicine effective in treating patients,1126,0.475133,535,12,0,0,0.0
3,is telehealth the future of healthcare 2,1210,0.456198,552,14,0,0,0.0
4,how people diverted to telehealth services and...,1141,0.449606,513,14,0,0,0.0
...,...,...,...,...,...,...,...,...
145,blockchain for payments,614,0.306189,188,13,0,0,0.0
146,the future of investing,1060,0.346226,367,17,0,0,0.0
147,big data analytics in healthcare,745,0.417450,311,11,0,0,0.0
148,business analytics in the healthcare industry,443,0.492099,218,15,0,0,0.0


In [52]:
final.to_excel('Output Data Structure.xlsx', encoding='utf-8')